# Part 1

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [281]:
import pandas as pd
import numpy as np
import lxml

In [282]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

In [283]:
df = pd.DataFrame(pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0])
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**

In [284]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
df.columns = column_names

**Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.**

In [285]:
df.drop(df[df['Borough'] == 'Not assigned'].index , inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.** http://queirozf.com/entries/pandas-dataframe-groupby-examples

In [286]:
df['Neighborhood'] = df['Neighborhood'].str.strip()
df_agg = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).to_frame()
df_agg.reset_index(inplace=True)
df_agg.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.**

In [287]:
df_agg.loc[df_agg['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_agg['Borough']

**In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.**

In [288]:
df_agg.shape

(103, 3)

# Part 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

**I used csv file that has the geographical coordinates of each postal code to create the dataframe**

In [289]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merging two dataframes together based on their Postal Code**

In [290]:
toronto_agg = pd.merge(df_agg, coordinates,how='left',left_on='PostalCode', right_on='Postal Code').drop('Postal Code', 1)

print(toronto_agg.shape)
toronto_agg.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. 

**Use geopy library to get the latitude and longitude values of Toronto**

In [291]:
from geopy.geocoders import Nominatim
import folium
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

In [292]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Create a map of Toronto with neighborhoods superimposed on top**

In [293]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_agg['Latitude'], toronto_agg['Longitude'], toronto_agg['Borough'], toronto_agg['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Define Foursquare Credentials and Version**

In [294]:
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
CLIENT_ID = input('CLIENT_ID: ')
CLIENT_SECRET = input('CLIENT_SECRET:')

Your credentails:


CLIENT_ID:  1
CLIENT_SECRET: 1


**Explore Neighborhoods**

In [295]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    null_venue_list=[] # neighborhoods without venues
    for name, lat, lng in zip(names, latitudes, longitudes):        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()
        
        try:
            results = results["response"]['groups'][0]['items']
        except:
            raise KeyError(results['meta']['errorDetail'])
            return
        
        if len(results) == 0:
            null_venue_list.append(name)
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    without_venues = pd.DataFrame(null_venue_list, columns=['Neighborhood'])
    
    return(nearby_venues, without_venues)

In [ ]:
toronto_venues, toronto_null_venue = getNearbyVenues(names=toronto_agg['Neighborhood'], 
                                                     latitudes=toronto_agg['Latitude'],
                                                     longitudes=toronto_agg['Longitude']
                                                     )
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

#### Analyze Each Neighborhood

In [ ]:
# one hot encoding
# Convert categorical variable into dummy/indicator variables
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
cols = list(toronto_onehot.columns.values) # make a list of all of the columns in the toronto_onehot
cols.pop(cols.index('Neighborhood')) # remove Neighborhood from list
toronto_onehot = toronto_onehot[['Neighborhood']+cols] # right order

print(toronto_onehot.shape)
toronto_onehot.head()

**Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [ ]:
# add to toronto_onehot values from toronto_null_venue
toronto_onehot = pd.concat([toronto_onehot, toronto_null_venue], ignore_index=True, sort=False).fillna(0)

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

**Get each neighborhood along with the top 10 most common venues in dataframe**

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood**

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Cluster Neighborhoods

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
# add clustering labels
neighborhoods_venues_s = neighborhoods_venues_sorted.copy()
neighborhoods_venues_s.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
toronto_merged = toronto_agg

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_s.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

**Visualize the resulting clusters**

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype('int32')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

In [ ]:
cols_for_print = [1, 2] + list(range(5, toronto_merged.shape[1]))

**Cluster 1 - Parks**

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[cols_for_print]]

**Cluster 2 - Big common cluster**

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[cols_for_print]]

**Cluster 3 - Small lonely cluster**

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[cols_for_print]]

**Cluster 4 - Park & Yoga Studio**

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[cols_for_print]]

**Cluster 5 - Baseball Field & Yoga Studio**

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[cols_for_print]]

*I tried to give all clusters meaningful names.
And some numbers about clusters: cluster 2 is the biggest, cluster 1 has 7 neighbourhoods, clusters 3 has just one neighbourhood, cluster 4 has 3 neighbourhoods and cluster 5 has 2 neighbourhoods.*